In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


###Let's sample

Suppose I have the population given by [5, 2, 3]. This means $N=10$, and we have three straetegies.

In [8]:
original_population = np.array([5, 2, 3], dtype=int)

I am interested figuring out what is the probability of sampling different groups of size 3. Since the population is finite, sampling is without replacement.

In [4]:
import itertools

In [6]:
group_size = 3
all_combinations = []
for i in itertools.product(range(0, group_size+1), repeat=3):
    if sum(i) == group_size:
        all_combinations.append(i)    

In [7]:
all_combinations

[(0, 0, 3),
 (0, 1, 2),
 (0, 2, 1),
 (0, 3, 0),
 (1, 0, 2),
 (1, 1, 1),
 (1, 2, 0),
 (2, 0, 1),
 (2, 1, 0),
 (3, 0, 0)]

Let's figure out by simulation, what is the change of sampling a group (1,0,2).

In [88]:
original_population = np.array([5, 2, 3], dtype=int)
group_size = 3

In [97]:
def sample_group(population, group_size):
    #basic check
    assert group_size <= np.sum(population), "Group has to be smaller than pop"
    #create a copy to mess around
    original_population = np.copy(np.array(population, dtype=int))
    number_of_strategies = len(original_population)
    #nothing sampled at begining
    sample = np.zeros_like(original_population)
    for _ in range(group_size):
        # determine "winner"
        position = np.random.choice(range(len(original_population)), 1, p=original_population/np.sum(original_population))[0] 
        # update sample and pop
        sample[position] = sample[position] +1 
        original_population[position] = original_population[position] -1    
    return sample

In [104]:
sample_group([5, 2, 3], 3)

array([2, 0, 1])

### Montecarlo!

In [105]:
from collections import Counter

In [150]:
pop = [5, 2, 3]
group_size = 3
number_of_samples = 500000
ans = []
for _ in range(number_of_samples):
    sample = tuple(sample_group(pop, group_size))
    ans.append(sample)
counter = Counter(ans)

In [151]:
for key in counter:
    print(key, end=" ")
    print(counter[key]/number_of_samples)

(1, 1, 1) 0.24991
(0, 2, 1) 0.025112
(2, 1, 0) 0.167432
(2, 0, 1) 0.248386
(0, 1, 2) 0.050312
(3, 0, 0) 0.084242
(1, 2, 0) 0.041256
(1, 0, 2) 0.12499
(0, 0, 3) 0.00836


###Now using probability theory
####*Hint*: Multivariate Hypergeometric

In [133]:
pop

[5, 2, 3]

In [142]:
target = (1, 0, 2)
target

(1, 0, 2)

In [143]:
import scipy as sp

In [154]:
def multivariate_hypergeometric(pop, target):
    N = sum(pop)
    n = sum(target)
    assert n <= N, "Give me something that makes sense!"
    assert len(pop) == len(target), "Give me something that makes sense!"
    product = 1.0
    for K_i, k_i in zip(pop, target):
        product = product*sp.misc.comb(K_i, k_i)
    return product/sp.misc.comb(N, n)

In [155]:
multivariate_hypergeometric(pop, target)

0.125